In [22]:
import dataset

from mxnet import autograd
from mxnet import gluon, image, nd, init, metric

import utils
from dataset import load_data_ChestX_ray14
from mxnet.gluon import nn
import matplotlib.pyplot as plt

In [93]:

train_iter, test_iter = load_data_ChestX_ray14(10, resize=224)



In [94]:

pretrained_net = gluon.model_zoo.vision.resnet50_v1(pretrained=True)
temp_data = nd.random_normal(shape=(1, 4096))
output_layer = nn.Dense(14,weight_initializer=init.Xavier()) 
output_layer.initialize()
# output_layer(temp_data)
output_layer
pretrained_net

ResNetV1(
  (features): HybridSequential(
    (0): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
    (2): Activation(relu)
    (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
    (4): HybridSequential(
      (0): BottleneckV1(
        (body): HybridSequential(
          (0): Conv2D(64 -> 64, kernel_size=(1, 1), stride=(1, 1))
          (1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
          (2): Activation(relu)
          (3): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
          (5): Activation(relu)
          (6): Conv2D(64 -> 256, kernel_size=(1, 1), stride=(1, 1))
          (7): BatchNorm(axis=1, eps=1e-05, m

In [95]:
tuning_net = nn.Sequential()
tuning_net.add(pretrained_net.features,
               output_layer)
tuning_net

temp_data = nd.random_normal(shape=(5, 3, 224, 224))
tuning_net(temp_data)


[[ 0.5101531  -1.5016284   1.0051639   0.17700872 -0.8343903   0.2240259
   0.32738596  0.48935762 -0.07724877 -1.5269042  -0.1039162   0.29450077
  -1.9516547  -0.32050657]
 [ 0.47053233 -1.4301226   1.028998    0.38350764 -0.99027884  0.17465699
   0.29515564  0.3994007   0.10518025 -1.3374596   0.00611044  0.17048465
  -1.9130688  -0.35607213]
 [ 0.65173924 -1.2484541   0.9028948   0.28644893 -1.0224499   0.03781413
   0.22287513  0.29557493  0.00702708 -1.5176992   0.04087946  0.28199685
  -1.8724945  -0.52502143]
 [ 0.3611023  -1.4016876   0.9190018   0.4726807  -1.111224    0.25888544
   0.47700617  0.39937437  0.1152609  -1.4652498  -0.15651976  0.44314834
  -1.8407826  -0.53066623]
 [ 0.5721691  -1.2066072   0.90031594  0.2753147  -1.2124552  -0.10998757
   0.05838182  0.38250345  0.2021772  -1.380657    0.29363713  0.3601156
  -1.7600687  -0.34499645]]
<NDArray 5x14 @cpu(0)>

In [98]:
loss_fn = gluon.loss.SoftmaxCrossEntropyLoss(sparse_label=False)

def timeit(fn):
    import time
    def decorator(*a, **b):
        start = time.time()
        fn(*a, **b)
        end = time.time()
        print(fn.__name__, 'take:', end - start, 's')
    return decorator
    
def evaluate(data_iter, net):
    if isinstance(data_iter, mx.io.MXDataIter):
        data_iter.reset()
    for X, y  in data_iter:
        output = net(X)
        loss = loss_fn(output, y)
    return loss.mean().asscalar() / len(test_iter)

In [97]:
import mxnet as mx
from dataset import load_data_ChestX_ray14

lr = 0.01
trainer = gluon.Trainer(tuning_net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = load_data_ChestX_ray14(32, resize=224)
for epoch in range(20):
    print('epoths', epoch, 'th:')

    for X, y in test_iter:
        print(X.shape)
        train_loss_sum = 0
        with autograd.record():
            output = tuning_net(X)
            loss = loss_fn(output, y)
        loss.backward()
        trainer.step(32)
        train_loss_sum += loss.mean().asscalar()
    test_loss = evaluate(test_iter, tuning_net)
    print("epoch %d, loss %.4f,  test acc %.3f" 
              % (epoch, train_loss_sum / len(train_iter),
                  test_loss))
        


epoths 0 th:
(32, 3, 224, 224)
(32, 3, 224, 224)
(19, 3, 224, 224)
epoch 0, loss 0.3961,  test acc 1.706
epoths 1 th:
(32, 3, 224, 224)
(32, 3, 224, 224)
(19, 3, 224, 224)
epoch 1, loss 0.2701,  test acc 1.650
epoths 2 th:
(32, 3, 224, 224)
(32, 3, 224, 224)
(19, 3, 224, 224)
epoch 2, loss 0.2374,  test acc 1.523
epoths 3 th:
(32, 3, 224, 224)
(32, 3, 224, 224)
(19, 3, 224, 224)
epoch 3, loss 0.1735,  test acc 1.532
epoths 4 th:
(32, 3, 224, 224)
(32, 3, 224, 224)
(19, 3, 224, 224)
epoch 4, loss 0.1446,  test acc 1.530
epoths 5 th:
(32, 3, 224, 224)
(32, 3, 224, 224)
(19, 3, 224, 224)
epoch 5, loss 0.1292,  test acc 1.432
epoths 6 th:
(32, 3, 224, 224)
(32, 3, 224, 224)
(19, 3, 224, 224)
epoch 6, loss 0.1171,  test acc 1.471
epoths 7 th:
(32, 3, 224, 224)
(32, 3, 224, 224)
(19, 3, 224, 224)
epoch 7, loss 0.1112,  test acc 1.370
epoths 8 th:
(32, 3, 224, 224)
(32, 3, 224, 224)
(19, 3, 224, 224)
epoch 8, loss 0.1128,  test acc 1.320
epoths 9 th:
(32, 3, 224, 224)
(32, 3, 224, 224)
(19, 3

KeyboardInterrupt: 

In [ ]:
train_iter, test_iter = load_data_ChestX_ray14(4, resize=224)
for data, label in train_iter:
    print(tuning_net(data))
    break

In [90]:
import mxnet as mx
import os

from sklearn import metrics

import dataset
import numpy as np
from mxnet import nd, gluon, autograd

from time import time

# ROOT_PATH = os.path.dirname(os.path.realpath(__file__))


# def download_ChestXray14(url=demo_url):
#     file_name = 'chestX-ray14.zip'
#     file_path = os.path.join(os.path.dirname(os.path.realpath(__file__)),
#                              file_name)
#     command = "wget --no-check-certificate '" + url + "' -O " + file_path
#     os.system("wget --no-check-certificate 'https://docs.google.com/uc?export=download&id"
#               "=12fbMgiEW0MuK85wXOWpZ4n_36llqz6Zw' -O chestX-ray14.zip")
#     os.system(command)
#     return file_path

def _get_batch(batch, ctx):
    """return features and labels on ctx"""
    if isinstance(batch, mx.io.DataBatch):
        features = batch.data[0]
        labels = batch.label[0]
    else:
        features, labels = batch
    return (gluon.utils.split_and_load(features, ctx),
            gluon.utils.split_and_load(labels, ctx),
            features.shape[0])


def evaluate_accuracy(data_iter, net, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc = nd.array([0])
    n = 0
    if isinstance(data_iter, mx.io.MXDataIter):
        data_iter.reset()
    for batch in data_iter:
        features, labels, batch_size = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y = y.astype('float32')
            acc += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc.wait_to_read()
    return acc.asscalar() / n


def evaluate_AUC(data_iter, net, ctx=[mx.cpu()]):
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    # TODO reset dsata_iter
    if isinstance(data_iter, mx.io.MXDataIter):
        print("yes!!!!!!!!")
    AUCs = np.zeros(shape=(dataset.num_labels, ))
    n = 0
    for batch in data_iter:
        # ATTENTION, this only works when the batch are equally split
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y_hat = net(X)
            y_hat = y_hat[:, :6]
            print('y_hat.shape:\t' ,y_hat.shape)
            print(y_hat[:5])
            y = y[:, :6]
            print('y.shape:\t', y.shape)
            print(y[:, :6])
            
            AUC = metrics.roc_auc_score(y_true=y.asnumpy(), y_score=y_hat.asnumpy(), average=None)
            print('AUC  :\t', AUC)
            AUCs += AUC

            n += 1

    # TODO same function with different implementation. test which one has the best perfermance.
        # y_hats = []
        # ys = []
        # for X, y in zip(features, labels):
        #     y_hats.append(net(X))
        #     ys.append(y)
        # nd.waitall()
        # for y_hat, ys in zip(y_hats, ys):
        #     AUCs += metrics.roc_auc_score(y_true=y.asnumpy(), y_score=y_hat.asnumpy(), average=None)

    return AUCs / n

In [99]:
import mxnet as mx
evaluate_AUC(test_iter, tuning_net, mx.cpu() )

y_hat.shape:	 (32, 6)

[[ 1.0468715   2.5704372   0.85869974  1.9694254   0.0546783  -0.40522003]
 [ 0.85632807  2.9594758   1.4306233   1.6067395  -0.17476259 -0.8086922 ]
 [ 0.22266877  2.2813516   1.7899029   1.8366884   0.28184253  0.37344462]
 [ 0.5947289   1.6204967   1.2425021   1.3474978  -0.36030486  0.29143634]
 [ 0.4341073   0.4807063   0.5747972   1.3299663   1.5840213   1.7985787 ]]
<NDArray 5x6 @cpu(0)>
y.shape:	 (32, 6)

[[0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 

ValueError: operands could not be broadcast together with shapes (14,) (6,) (14,) 

In [66]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 0, 1, 1])
ys = np.vstack([y_true  for i in range(5)])
ys = ys.T

y_scores = np.array([0.1, 0.4, 0.35, 0.8])
y_scos = np.vstack([y_scores for i in range(5)])
y_scos = y_scos.T
print(y_scos)
ys[0, 4] = 1
ys[1, 0:3] = 1
# ys[3,2] = 0
print(ys)
roc_auc_score(y_true= ys , y_score=y_scos,average=None)

[[0.1  0.1  0.1  0.1  0.1 ]
 [0.4  0.4  0.4  0.4  0.4 ]
 [0.35 0.35 0.35 0.35 0.35]
 [0.8  0.8  0.8  0.8  0.8 ]]
[[0 0 0 0 1]
 [1 1 1 0 0]
 [1 1 1 1 1]
 [1 1 1 1 1]]


array([1.        , 1.        , 1.        , 0.75      , 0.33333333])

In [39]:
ys

array([[0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])